In [ ]:
# web_scraper.py

# Installationsbefehle (entferne die Kommentarzeichen, wenn du sie ausführen möchtest)
# pip install nest_asyncio playwright beautifulsoup4 fpdf

import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from fpdf import FPDF
import logging
from pathlib import Path
import json

# Damit wir async in Jupyter/Colab verwenden können
nest_asyncio.apply()

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class WebScraper:
    def __init__(self, url, output_file="extracted_teaser_data", output_format="txt", output_dir="data"):
        """
        Initialisiert die WebScraper-Klasse.

        :param url: URL der zu scrapenden Webseite
        :param output_file: Basisname der Ausgabedatei ohne Dateiendung
        :param output_format: Format der Ausgabedatei ('txt', 'pdf', 'json', etc.)
        :param output_dir: Verzeichnis, in dem die Ausgabedateien gespeichert werden sollen
        """
        self.url = url
        self.output_file = output_file
        self.output_format = output_format.lower()
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)  # Erstelle den Ausgabeordner, falls er nicht existiert

    async def fetch_page_content(self):
        """
        Lädt die Webseite und gibt den HTML-Inhalt zurück.

        :return: HTML-Inhalt der Webseite
        """
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch(headless=True)
                page = await browser.new_page()

                # Webseite laden
                logging.info(f"Lade Webseite: {self.url}")
                await page.goto(self.url)

                # Warte, bis die Seite vollständig geladen ist
                await page.wait_for_load_state("networkidle")

                # Gesamten HTML-Inhalt der Seite erfassen
                content = await page.content()

                # Browser schließen
                await browser.close()

                logging.info("Webseiteninhalt erfolgreich abgerufen.")
                return content
        except Exception as e:
            logging.error(f"Fehler beim Abrufen der Seite: {e}")
            return None

    # Beautiful
    def extract_important_attributes(self, html_content):
        """
        Extrahiert wichtige Attribute aus dem HTML-Inhalt.

        :param html_content: HTML-Inhalt der Webseite
        :return: Liste der extrahierten Daten
        """
        try:
            soup = BeautifulSoup(html_content, "html.parser")
            data = []

            # Suche nach spezifischen Elementen, die die relevanten Attribute enthalten
            for element in soup.find_all(attrs={"teaser-title": True, "teaser-description": True, "teaser-url": True}):
                # Extrahiere die Werte der gewünschten Attribute
                teaser_title = element.get("teaser-title")
                teaser_description = element.get("teaser-description")
                teaser_url = element.get("teaser-url")

                # Füge die extrahierten Daten zur Liste hinzu
                data.append({"title": teaser_title, "description": teaser_description, "url": teaser_url})

            logging.info("Wichtige Attribute erfolgreich extrahiert.")
            return data
        except Exception as e:
            logging.error(f"Fehler beim Extrahieren der Attribute: {e}")
            return []



    def save_to_txt(self, data):
        """
        Speichert die Daten als Textdatei.

        :param data: Liste der zu speichernden Daten
        """
        try:
            file_path = self.output_dir / f"{self.output_file}.txt"
            if file_path.exists():
                logging.warning(f"Die Datei '{file_path}' existiert bereits. Speichern wird übersprungen.")
                return

            with file_path.open("w", encoding="utf-8") as file:
                for item in data:
                    file.write(f"Title: {item['title']}\n")
                    file.write(f"Description: {item['description']}\n")
                    file.write(f"URL: {item['url']}\n")
                    file.write("\n")  # Leerzeile zwischen den Einträgen

            logging.info(f"Daten wurden erfolgreich in '{file_path}' gespeichert.")
        except Exception as e:
            logging.error(f"Fehler beim Speichern der Daten als TXT: {e}")



    def save_to_pdf(self, data):
        """
        Speichert die Daten als PDF-Datei.

        :param data: Liste der zu speichernden Daten
        """
        try:
            file_path = self.output_dir / f"{self.output_file}.pdf"
            if file_path.exists():
                logging.warning(f"Die Datei '{file_path}' existiert bereits. Speichern wird übersprungen.")
                return

            pdf = FPDF()
            pdf.add_page()

            # Füge die Unicode-fähige Schriftart hinzu
            pdf.add_font("DejaVu", "", "fonts/DejaVuSans.ttf", uni=True)  # Stelle sicher, dass DejaVuSans.ttf im fonts/ Ordner ist
            pdf.set_font("DejaVu", "", 12)

            for item in data:
                title = item['title'] if item['title'] else ''
                description = item['description'] if item['description'] else ''
                url = item['url'] if item['url'] else ''

                pdf.cell(200, 10, txt=f"Title: {title}", ln=True)
                pdf.multi_cell(0, 10, txt=f"Description: {description}")
                pdf.cell(200, 10, txt=f"URL: {url}", ln=True)
                pdf.ln(5)  # Leerzeile zwischen den Einträgen

            pdf.output(str(file_path))

            logging.info(f"Daten wurden erfolgreich in '{file_path}' gespeichert.")
        except Exception as e:
            logging.error(f"Fehler beim Speichern der Daten als PDF: {e}")



    def save_to_json(self, data):
        """
        Speichert die Daten als JSON-Datei.

        :param data: Liste der zu speichernden Daten
        """
        try:
            file_path = self.output_dir / f"{self.output_file}.json"
            if file_path.exists():
                logging.warning(f"Die Datei '{file_path}' existiert bereits. Speichern wird übersprungen.")
                return

            with file_path.open("w", encoding="utf-8") as file:
                json.dump(data, file, ensure_ascii=False, indent=4)

            logging.info(f"Daten wurden erfolgreich in '{file_path}' gespeichert.")
        except Exception as e:
            logging.error(f"Fehler beim Speichern der Daten als JSON: {e}")



    def save_data_to_file(self, data):
        """
        Speichert die Daten im angegebenen Format.

        :param data: Liste der zu speichernden Daten
        """
        if self.output_format == "txt":
            self.save_to_txt(data)
        elif self.output_format == "pdf":
            self.save_to_pdf(data)
        elif self.output_format == "json":
            self.save_to_json(data)
        else:
            logging.warning(f"Unbekanntes Ausgabeformat '{self.output_format}'. Daten werden nicht gespeichert.")



    async def run(self):
        """
        Führt den gesamten Scraping-Prozess aus.
        """
        # HTML-Inhalt abrufen
        content = await self.fetch_page_content()
        if content is None:
            logging.error("Keine Daten zum Verarbeiten. Beende das Programm.")
            return

        # Wichtige Attribute extrahieren
        extracted_data = self.extract_important_attributes(content)
        if not extracted_data:
            logging.warning("Keine Daten extrahiert.")
            return

        # Daten speichern
        self.save_data_to_file(extracted_data)


# Alternative

In [ ]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
import os

# Damit wir async in Jupyter/Colab verwenden können
nest_asyncio.apply()

async def fetch_and_save_pdf(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Webseite laden
        await page.goto(url)

        # Warte, bis die Seite vollständig geladen ist
        await page.wait_for_load_state("networkidle")

        # Erstelle einen gültigen Dateinamen aus der URL
        filename = url.replace("http://", "").replace("https://", "").replace("/", "_").replace(":", "_")
        filename = f"{filename}.pdf"
        filepath = os.path.join(os.getcwd(), filename)

        # Speichere die Seite als PDF
        await page.pdf(path=filepath, format="A4")
        print(f"PDF gespeichert: {filepath}")

        # Browser schließen
        await browser.close()

async def main():
    urls = [
        #"http://regelwerke.vbg.de/vbg_gese/nagg/nagg_0_.html",
        "http://regelwerke.vbg.de/vbg_gese/narbgg/narbgg_0_.html",
        "http://regelwerke.vbg.de/vbg_gese/narbschg/narbschg_0_.html",
        "http://regelwerke.vbg.de/vbg_gese/narbzg/narbzg_0_.html",
        "http://regelwerke.vbg.de/vbg_gese/nasig/nasig_0_.html",
        "http://regelwerke.vbg.de/vbg_gese/naueg/naueg_0_.html"

    ]

    # Erstelle eine Liste von Aufgaben
    tasks = [fetch_and_save_pdf(url) for url in urls]

    # Führe die Aufgaben gleichzeitig aus
    await asyncio.gather(*tasks)

# Führe das Hauptprogramm aus
asyncio.run(main())